In [2]:
import json

In [63]:
with open("./reports/example.json", 'r') as f:
    data = json.load(f)
data

{'rows': [{'druid': 'druid:rx212yq2993',
   'errors': {},
   'counts': {'total_size': 1530211,
    'mimetypes': {'image/jpeg': 2},
    'filename_no_extension': 0}},
  {'druid': 'druid:xk191tr8945',
   'errors': {'empty_object': True, 'missing_files': True},
   'counts': {'total_size': 0, 'mimetypes': {}, 'filename_no_extension': 0}},
  {'druid': 'druid:sz766yw0944',
   'errors': {},
   'counts': {'total_size': 1448949,
    'mimetypes': {'image/jpeg': 2},
    'filename_no_extension': 0}},
  {'druid': 'druid:sh143xn6642',
   'errors': {},
   'counts': {'total_size': 3308587,
    'mimetypes': {'image/jpeg': 2},
    'filename_no_extension': 0}},
  {'druid': 'druid:xg322sp9596',
   'errors': {'dupes': True},
   'counts': {'total_size': 6953821,
    'mimetypes': {'image/jpeg': 6},
    'filename_no_extension': 0}},
  {'druid': 'druid:gn012km8355',
   'errors': {},
   'counts': {'total_size': 2975415,
    'mimetypes': {'image/jpeg': 2},
    'filename_no_extension': 0}},
  {'druid': 'druid:oo00

In [64]:
rows = data.pop('rows')
rows

[{'druid': 'druid:rx212yq2993',
  'errors': {},
  'counts': {'total_size': 1530211,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:xk191tr8945',
  'errors': {'empty_object': True, 'missing_files': True},
  'counts': {'total_size': 0, 'mimetypes': {}, 'filename_no_extension': 0}},
 {'druid': 'druid:sz766yw0944',
  'errors': {},
  'counts': {'total_size': 1448949,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:sh143xn6642',
  'errors': {},
  'counts': {'total_size': 3308587,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:xg322sp9596',
  'errors': {'dupes': True},
  'counts': {'total_size': 6953821,
   'mimetypes': {'image/jpeg': 6},
   'filename_no_extension': 0}},
 {'druid': 'druid:gn012km8355',
  'errors': {},
  'counts': {'total_size': 2975415,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:oo000oo0000',
  'errors': {'missing_files

In [65]:
summary = data.pop('summary')
summary['total number of objects'] = len(rows)
summary

{'objects_with_error': 3,
 'mimetypes': {'image/jpeg': 1320},
 'start_time': '2021-10-18 12:03:45 -0700',
 'total_size': 1108060670,
 'total number of objects': 7}

In [205]:
import pandas as pd

In [66]:
dfSummary = pd.DataFrame(summary)
dfSummary

,objects_with_error,mimetypes,start_time,total_size,total number of objects
image/jpeg,3,1320,2021-10-18 12:03:45 -0700,1108060670,7


In [67]:
dfRows = pd.DataFrame(rows)
dfRows

,druid,errors,counts
0,druid:rx212yq2993,{},"{'total_size': 1530211, 'mimetypes': {'image/j..."
1,druid:xk191tr8945,"{'empty_object': True, 'missing_files': True}","{'total_size': 0, 'mimetypes': {}, 'filename_n..."
2,druid:sz766yw0944,{},"{'total_size': 1448949, 'mimetypes': {'image/j..."
3,druid:sh143xn6642,{},"{'total_size': 3308587, 'mimetypes': {'image/j..."
4,druid:xg322sp9596,{'dupes': True},"{'total_size': 6953821, 'mimetypes': {'image/j..."
5,druid:gn012km8355,{},"{'total_size': 2975415, 'mimetypes': {'image/j..."
6,druid:oo000oo0000,{'missing_files': True},"{'total_size': 666, 'mimetypes': {'image/jpeg'..."


In [75]:
with open("./reports/example.json", 'r') as f:
    dfRowsFlattened = pd.json_normalize(json.load(f), 'rows')
dfRowsFlattened

,druid,counts.total_size,counts.mimetypes.image/jpeg,counts.filename_no_extension,errors.empty_object,errors.missing_files,errors.dupes
0,druid:rx212yq2993,1530211,2.0,0,NaN,NaN,NaN
1,druid:xk191tr8945,0,NaN,0,True,True,NaN
2,druid:sz766yw0944,1448949,2.0,0,NaN,NaN,NaN
3,druid:sh143xn6642,3308587,2.0,0,NaN,NaN,NaN
4,druid:xg322sp9596,6953821,6.0,0,NaN,NaN,True
5,druid:gn012km8355,2975415,2.0,0,NaN,NaN,NaN
6,druid:oo000oo0000,666,1.0,0,NaN,True,NaN


In [76]:
dfRowsFlattened.set_index('druid', inplace=True)
dfRowsFlattened

,counts.total_size,counts.mimetypes.image/jpeg,counts.filename_no_extension,errors.empty_object,errors.missing_files,errors.dupes
druid,,,,,,
druid:rx212yq2993,1530211,2.0,0,NaN,NaN,NaN
druid:xk191tr8945,0,NaN,0,True,True,NaN
druid:sz766yw0944,1448949,2.0,0,NaN,NaN,NaN
druid:sh143xn6642,3308587,2.0,0,NaN,NaN,NaN
druid:xg322sp9596,6953821,6.0,0,NaN,NaN,True
druid:gn012km8355,2975415,2.0,0,NaN,NaN,NaN
druid:oo000oo0000,666,1.0,0,NaN,True,NaN


In [206]:
err_cols = {
    "error": ["fake", "fake"],
    "druids": ["a", "b"]
}
dfErrors = pd.DataFrame(err_cols)
dfErrors

TypeError: 'list' object is not callable

In [189]:
errors = []
errors.append('foo')
errors.append('bar')
errors

['foo', 'bar']

In [192]:
d = {}
d['a'] = ['b']
d['c'] = ['d', 'e']
d

{'a': ['b'], 'c': ['d', 'e']}

In [204]:
possible_errors = [
    'dor_connection_error', 
    'dupes', 
    'empty_files', 
    'empty_object', 
    'filename_no_extension',  
    'item_not_registered', 
    'missing_files',
    'empty_manifest',
    'files_found_mismatch',
    'missing_media_container_name_or_manifest'
     ]
errs2druids = {}
for err in possible_errors:
    err_col = "errors.% s" % err
    if ((err_col) in dfRowsFlattened.columns):
        err_druids = (dfRowsFlattened[(dfRowsFlattened[err_col] == True)]).index.tolist()
        # errs2druids[err] = ','.join(err_druids)
        errs2druids[err] = err_druids
errs2druids
dfErrors = pd.DataFrame(errs2druids)
# dfErrors.set_index('error', inplace=True)
# dfErrors

TypeError: 'list' object is not callable

In [174]:
if 'errors.empty_files' in dfRowsFlattened.columns:
    print("yes")

In [96]:
dupes = dfRowsFlattened[(dfRowsFlattened['errors.dupes'] == True)].index.tolist()
dupes

['druid:xg322sp9596']

In [107]:
dfRowsFlattened.keys().array.contains('errors.dupes')

AttributeError: 'PandasArray' object has no attribute 'contains'

In [113]:
if 'errors.empty_files' in dfRowsFlattened.columns:
    emptyFiles = dfRowsFlattened[(dfRowsFlattened['errors.empty_files'] == True)].index.tolist()
else:
    emptyFiles = []
emptyFiles

[]

In [90]:
emptyObjects = dfRowsFlattened[(dfRowsFlattened['errors.empty_object'] == True)].index.tolist()
emptyObjects

['druid:xk191tr8945']

In [92]:
missingFiles = dfRowsFlattened[(dfRowsFlattened['errors.missing_files'] == True)].index.tolist()
missingFiles

['druid:xk191tr8945', 'druid:oo000oo0000']

['druid:xg322sp9596']

In [146]:
errors.

,druid,errors.empty_object,errors.missing_files,errors.dupes
1,druid:xk191tr8945,True,True,NaN
